In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import seaborn as sns
import gzip

In [2]:
provided_data_root_path = '~/Documents/beagle/provided_data'
provided_data_input_path = '~/Documents/beagle/inputs/'
provided_data_output_path = '~/Documents/beagle/outputs/'

## PULSAR File Generation

In [ ]:
maf_vals = pd.read_csv('~/downloads/Runcie_Resource_Allocation/new_allele_frequencies.csv')
genetic_map_provided = pd.read_csv('20A823_Genetic_Map.csv').drop(columns = ['Unnamed: 0'])
cam_map_provided = pd.read_csv('camMap.csv').drop(columns = ['Unnamed: 0'])
ped = pd.read_csv('~/Downloads/Runcie_Resource_Allocation/pulsar_pedigree_input.csv')

In [ ]:
def get_vcf_names(vcf_path):
    with gzip.open(vcf_path, "rt") as ifile:
        for line in ifile:
            if line.startswith("#CHROM"):
                vcf_names = [x for x in line.split('\t')]
                break
    ifile.close()
    return vcf_names

def find_parents(curr_child):
    family_tree = []
    if curr_child not in names:
        return family_tree
        
    q = []
    visited = {curr_child: True}
    q.append(curr_child)
    while q:
        v = q.pop(0)
        if v != '0': family_tree.append(v)
        results = both_parents_genotyped.loc[both_parents_genotyped.ID == v]
        if not len(results):
            continue
        row = results.iloc[0]
        mother, father = row['MO'], row['FA']
        if mother == '0' and father == '0':
            continue
        elif mother != '0' and father != '0':
            q.append(father)
            visited[father] = True
            q.append(mother)
            visited[mother] = True
        elif mother != '0':
            q.append(father)
            visited[father] = True
        elif father != '0':
            q.append(mother)
            visited[mother] = True
    return list(set(family_tree))

def draw_ancestry_network():
    # create all edge tuples
    g = nx.DiGraph()
    g.add_edges_from(edges_selected)
    g.add_nodes_from(nodes_selected)

    plt.figure(figsize = (15, 12))
    pos = nx.spring_layout(g)
    nx.draw_networkx_nodes(g, pos, cmap = plt.get_cmap('summer'), node_color = '#ADD8E6', node_size = 500)
    nx.draw_networkx_labels(g, pos)
    nx.draw_networkx_edges(g, pos, arrows = True)
    plt.show()
    
def find_children(nodename):
    conn = [v for u, v in nx.bfs_edges(g, nodename)]
    return conn

def find_lineages(founder, offspring):
    curr_lineages = []
    for o in offspring:
        curr_lineages.append('_'.join(nx.shortest_path(g, source = founder, target = o)))
    return curr_lineages

In [ ]:
names = get_vcf_names(provided_data_root_path + '')[9:]
print(len(names))
names[:10]

In [ ]:
all_parents = list(set(list(ped['FA']) + list(ped['MO'])) - set('0'))
parent_vcf_intersect = list(set(all_parents).intersection(set(names)))

In [ ]:
# find all organisms in the pedigree table s.t. both parents are genotyped

both_parents_genotyped = ped.loc[ped['FA'].isin(parent_vcf_intersect) & ped['MO'].isin(parent_vcf_intersect)]
print(both_parents_genotyped.shape)
both_parents_genotyped.head()

In [ ]:
nodes_selected = find_parents('Organism11190')
edges_selected = []

for _, i in both_parents_genotyped.iterrows():
    child, p1, p2 = i['ID'], i['FA'], i['MO']
    if child in nodes_selected and p1 != '0' and p2 != '0':
        edges_selected.append((p1, child))
        edges_selected.append((p2, child))
        
        if p1 not in nodes_selected: nodes_selected.append(p1)
        if p2 not in nodes_selected: nodes_selected.append(p2)

In [ ]:
# generate founder hash & lineage
founderhash = {}
for f in new_founders:
    founderhash[f] = find_children(f)
    
print(founderhash)
    
founder_data = {}
founder_data['founders'] = list(founderhash.keys())
founder_data['offspring'] = [':'.join(i) for i in list(founderhash.values())]
founder_df = pd.DataFrame.from_dict(founder_data)
founder_df.head()

founder_df.to_csv('vcf_split_method/founderhash.csv', index = False)

all_lineages = []
for f, o in zip(founder_data['founders'], founder_data['offspring']):
    all_lineages.append(f)
    if len(o.split(':')) and o.strip() != "":
        all_lineages.extend(find_lineages(f, o.split(':')))
        
print(all_lineages[:10], len(all_lineages))

lineage_df = pd.DataFrame({'lineage': all_lineages})
lineage_df.to_csv('vcf_split_method/pulsarlineage.csv', index = False)

## BEAGLE File Generation